## 4.1.6 재현 신경망(Recurrent Neural Network) 분류 모델

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import json

## 학습 데이터 파일 로드

In [ ]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

INPUT_TRAIN_DATA_FILE_NAME = 'train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'train_label.npy'
DATA_CONFIGS_FILE_NAME = 'data_configs.json'

In [ ]:
input_data = np.load(open(DATA_IN_PATH + INPUT_TRAIN_DATA_FILE_NAME, 'rb'))
label_data = np.load(open(DATA_IN_PATH + LABEL_TRAIN_DATA_FILE_NAME, 'rb'))
prepro_configs = None

with open(DATA_IN_PATH + DATA_CONFIGS_FILE_NAME, 'r') as f:
    prepro_configs = json.load(f)
    print(prepro_configs.keys())

In [ ]:
TEST_SPLIT = 0.1
RNG_SEED = 13371447

input_train, input_test, label_train, label_test = train_test_split(input_data, label_data, 
                                                                    test_size=TEST_SPLIT, random_state=RNG_SEED)

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 10

def mapping_fn(X, Y):
    input, label = {'x': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_test, label_test))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

## 모델 정의

In [ ]:
vocabulary_size = prepro_configs['vocab_size']
embedding_size = 100
hidden_state_dim = 150
hidden_layer_dim = 150

learning_rate = 0.001


def model_fn(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    input_layer = tf.keras.layers.Embedding(
                    VOCAB_SIZE,
                    EMB_SIZE,
                    initializer=params['embedding_initializer']
                    )(features['x'])
    
    training = False
    if TRAIN:
        training = True
    
    input_layer = tf.keras.layers.Dropout(0.2training)(embedding_layer)
    
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [hidden_state_dim, hidden_state_dim]]
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

    outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                       inputs=input_layer,
                                       dtype=tf.float32)
    
    hidden_layer = tf.keras.layers.Dense(hidden_layer_dim, activation=tf.nn.tanh)(outputs[:,-1,:])
    hidden_layer = tf.keras.layers.Dropout(0.2)(hidden_layer)
    logits = tf.keras.layers.Dense(2)(hidden_layer)
    
    if PREDICT:
        softmax_logits = tf.nn.softmax(logits)
        
        predictions = {'sentiment': tf.argmax(softmax_logits, 1),
                       'confidence': tf.reduce_max(softmax_logits, axis=1)}
        
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  predictions=predictions)
    
    one_hot_labels = tf.one_hot(labels, 2)
    loss = tf.losses.softmax_cross_entropy(one_hot_labels, logits)
    
    elif EVAL:
        accuracy = tf.metrics.accuracy(labels, tf.argmax(logits, 1))
        eval_metric_ops = {'acc': accuracy}

        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    elif TRAIN:
        global_step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  train_op=train_op,
                  loss=loss)

In [ ]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

est = tf.estimator.Estimator(model_fn=model_fn,
                             model_dir=DATA_OUT_PATH + 'checkpoint')

In [ ]:
est.train(train_input_fn)

In [ ]:
est.evaluate(eval_input_fn)

## 캐글 평가 데이터셋 만들기

In [ ]:
INPUT_TEST_DATA_FILE_NAME = 'test_input.npy'

test_input_data = np.load(open(DATA_IN_PATH + INPUT_TEST_DATA_FILE_NAME, 'rb'))

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"text":test_input_data}, shuffle=False)

In [ ]:
predictions = np.array([p['sentiment'] for p in est.predict(input_fn=
predict_input_fn)])

In [ ]:
DEFAULT_PATH='~/.kaggle/competitions/word2vec-nlp-tutorial/'

test = pd.read_csv(DEFAULT_PATH + "testData.tsv", header=0, delimiter="\t", quoting=3 )

In [ ]:
test.head(10)

In [ ]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

output = pd.DataFrame( data={"id":test["id"], "sentiment":list(predictions)} )
output.to_csv(DATA_OUT_PATH + "rnn_predict13.csv", index=False, quoting=3 )